In [1]:
import pandas as pd
import time
import random
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import math

In [2]:
df = pd.read_json("job_postings_prepped.json")

In [3]:
import pickle
vectorizer = pickle.load(open('vectorizer.pkl','rb'))

## LSH **Calculations**


In [6]:
import json

k = 300 #TODO: find out optimal value
vector_dimensions = len(df['Vector'][0])
non_zero_values = len(df['Vector'][0]) #TODO: find out optimal value

def generate_hash_function(k,vector_dimensions,non_zero_values):
    hash_functions_list = []

    for i in range(k):
        if i%10 == 0: print("Progress:", str(i)+"/"+str(k))
        hash_functions = np.zeros(vector_dimensions)
        selected_indexes = random.sample(range(vector_dimensions),non_zero_values)
        
        for index in selected_indexes:
            random_vector = np.random.randn(1)
            hash_functions[index] = random_vector

        hash_functions_list.append(hash_functions)

    return hash_functions_list

random_hash_functions = generate_hash_function(k,vector_dimensions,non_zero_values)
random_hash_functions_df = pd.DataFrame({'Hash_Functions': random_hash_functions})

random_hash_functions_df.to_json('LSH Search Results/LSH 300 Random Hash Functions.json', index=False)


Progress: 0/300


C:\Users\Admin\AppData\Local\Temp\ipykernel_4364\4081861469.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hash_functions[index] = random_vector


Progress: 10/300
Progress: 20/300
Progress: 30/300
Progress: 40/300
Progress: 50/300
Progress: 60/300
Progress: 70/300
Progress: 80/300
Progress: 90/300
Progress: 100/300
Progress: 110/300
Progress: 120/300
Progress: 130/300
Progress: 140/300
Progress: 150/300
Progress: 160/300
Progress: 170/300
Progress: 180/300
Progress: 190/300
Progress: 200/300
Progress: 210/300
Progress: 220/300
Progress: 230/300
Progress: 240/300
Progress: 250/300
Progress: 260/300
Progress: 270/300
Progress: 280/300
Progress: 290/300


In [4]:
# import time

# Perform dot product and get binary vectors
def generate_binary_vectors(input_vector,random_hash_functions_df):
    binary_vector = []
    for i,row in random_hash_functions_df.iterrows():
        input_vector = np.asarray(input_vector, dtype=np.float64)
        hash_function = np.asarray(row['Hash_Functions'], dtype=np.float64)
    
        dot_product = np.dot(input_vector,hash_function)
        # result = 0 if dot_product <= 0 else 1
        result = np.where(dot_product <= 0, 0, 1)
        binary_vector.append(result.tolist())
    return binary_vector

# Make a new column called Binary Vectors for all the binary vectors
# random_hash_functions_df = pd.read_json('LSH Search Results/LSH Random Hash Functions.json')
# df['Binary Vectors'] = df['Vector'].apply(lambda x : generate_binary_vectors(x,random_hash_functions_df))

In [56]:
import json
test_hash = pd.DataFrame({'Hash_Functions': generate_hash_function(2, 105248, 105248)})
generate_binary_vectors(json.loads(df.iloc[0]["Vector"]), test_hash)
# len(query_matrix[0])
# json.loads(df.iloc[0]["Vector"])

C:\Users\Admin\AppData\Local\Temp\ipykernel_14292\3669591667.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  hash_functions[index] = random_vector


Time Taken to generate binary vectors: 0.003000020980834961


[0, 1]

In [23]:
# Perform linear search on binary vector using hamming distance Dh
def hamming_distance(query_vector,database_sample):
  mismatched_bits = 0
  for i in range(len(query_vector)):
    if query_vector[i] != database_sample[i]:
      mismatched_bits += 1
  return mismatched_bits

##### Find Best K Value Time from 10 to 100

In [ ]:
#Calculate K Value Time Comparison
# k_time_dict = {}
# vector_dimensions = len(df['Vector'][0])
# non_zero_values = len(df['Vector'][0]) #TODO: find out optimal value

# for k in range(10,101,10):
#     start = time.time()

#     random_hash_functions = generate_hash_function(k,vector_dimensions,non_zero_values)
#     binary_vectors_list = df['Vector'].apply( lambda x : one_hashing_function(x,random_hash_functions))

#     end = time.time()
#     results_time = end - start
#     k_time_dict[k] = results_time
#     k_df = pd.DataFrame(list(k_time_dict.items()), columns=['K Value', 'Time'])
#     k_df.to_csv(f'LSH Search Results/K_Value_Time.csv', index=False)


### Calculate binary of dataset

In [5]:
import time
import json

# Import Hash functions *
random_hash_functions_df = pd.read_json('LSH Search Results/LSH 300 Random Hash Functions.json')
print("Imported  hash functions from Json file")
# df['Vector'] = df['Vector'].apply(lambda x : json.loads(x))

for i in range(10,300,20):
    start_time = time.time()
    i_size_random_hash_functions = random_hash_functions_df.sample(i)
    print("Index:",list(i_size_random_hash_functions.index))
    # # Do LSH Calculations for dataset *
    df['Binary Vectors'] = df['Vector'].apply( lambda x : generate_binary_vectors(x,i_size_random_hash_functions))
    end_time = time.time()
    print("Time Taken to generate binary vectors(k =",str(i)+"):", end_time-start_time, "secs")
    print("Binary vectors successfully added, Exporting to Json...")
    df.to_csv('LSH Search Results/LSH Database with Binary Vectors (k is '+str(i)+').csv')

Imported  hash functions from Json file
Index: [159, 215, 144, 31, 0, 33, 210, 299, 227, 133]
Time Taken to generate binary vectors(k = 10): 340.71075892448425 secs
Binary vectors successfully added, Exporting to Json...
Index: [56, 59, 46, 43, 84, 227, 172, 5, 165, 52, 92, 213, 176, 17, 67, 77, 96, 131, 175, 110, 254, 41, 66, 290, 287, 58, 6, 219, 4, 32]
Time Taken to generate binary vectors(k = 30): 935.3069818019867 secs
Binary vectors successfully added, Exporting to Json...
Index: [243, 60, 115, 254, 59, 296, 19, 299, 186, 155, 272, 257, 141, 54, 149, 266, 209, 193, 225, 221, 287, 146, 293, 196, 75, 67, 276, 175, 295, 142, 104, 22, 49, 114, 42, 203, 58, 6, 292, 55, 108, 82, 286, 217, 69, 268, 91, 48, 30, 93]
Time Taken to generate binary vectors(k = 50): 1529.8507180213928 secs
Binary vectors successfully added, Exporting to Json...
Index: [8, 53, 289, 133, 226, 21, 216, 295, 268, 296, 214, 165, 218, 61, 121, 243, 62, 140, 16, 143, 183, 223, 262, 30, 55, 110, 284, 91, 89, 77, 95, 

## Calculate 10 different queries accuracies

In [6]:
# Calculate 10 different queries accuracies
import os

# Defining queries
query_dict = {'Q1': "resilient investment banker", 
              'Q2': "2 years experience product manager", 
              'Q3': "10 years risk analyst problem solver", 
              'Q4': "tax analyst for big company", 
              'Q5': "software engineer for google or amazon", 
              'Q6': "video editor for advertisements with 5 year experience",
              'Q7': "full time senior head nurse position",
              'Q8': "after school math and science tutor",
              'Q9': "dietitian for professional atheletes",
              'Q10': "costume designer and makeup artist"}

query_df = pd.DataFrame.from_dict(query_dict, orient='index', columns=['Query Text'])

# Using the imported count vectorizer to generate counter vector of queries (N)
query_vectors = vectorizer.transform(list(query_dict.values()))
query_matrix = query_vectors.toarray()

# Import Hash functions *
random_hash_functions_df = pd.read_json('LSH Search Results/LSH Random Hash Functions.json')

# Do LSH Calculations for query vectors *
query_dict_binary_vectors = [generate_binary_vectors(vector, random_hash_functions_df) for vector in query_matrix]

query_df['Vectors'] = [row.tolist() for row in query_matrix]
query_df['Binary Vectors'] = query_dict_binary_vectors

query_df.to_csv('LSH Search Results/LSH Query Vectors with Binary Vectors.csv')

Time Taken to generate binary vectors: 0.18794512748718262
Time Taken to generate binary vectors: 0.1875159740447998
Time Taken to generate binary vectors: 0.18975400924682617
Time Taken to generate binary vectors: 0.18224525451660156
Time Taken to generate binary vectors: 0.17214012145996094
Time Taken to generate binary vectors: 0.20138168334960938
Time Taken to generate binary vectors: 0.180006742477417
Time Taken to generate binary vectors: 0.1985175609588623
Time Taken to generate binary vectors: 0.19063329696655273
Time Taken to generate binary vectors: 0.19053316116333008


In [ ]:
# Calculate 10 different queries accuracies
import os

# Defining queries
query_dict = {'Q1': "resilient investment banker", 
              'Q2': "2 years experience product manager", 
              'Q3': "10 years risk analyst problem solver", 
              'Q4': "tax analyst for big company", 
              'Q5': "software engineer for google or amazon", 
              'Q6': "video editor for advertisements with 5 year experience",
              'Q7': "full time senior head nurse position",
              'Q8': "after school math and science tutor",
              'Q9': "dietitian for professional atheletes",
              'Q10': "costume designer and makeup artist"}

query_df = pd.DataFrame.from_dict(query_dict, orient='index', columns=['Query Text'])

# Combine the data from the DataFrame and query_dict into a single list
# all_data = list(df["Data"].array) + list(query_dict.values())

# vectorizer = CountVectorizer()
# matrix = vectorizer.fit_transform(list(query_dict.values()))
# matrix = matrix.toarray()

# Using the imported count vectorizer to generate counter vector of queries (N)
query_vectors = vectorizer.transform(list(query_dict.values()))
query_matrix = query_vectors.toarray()

# Separate the vectors for the DataFrame and query_dict *
# df_vectors = matrix[:len(df)]
# query_dict_vectors = matrix[len(df):]

# df["Vector"] = [row.tolist() for row in df_vectors]

# Variables for LSH #TODO: EDIT THIS FOR ACCURACY *
k = 100
vector_dimensions = len(df['Vector'][0])
non_zero_values = len(df['Vector'][0])

# Generate Random Hash Functions
# random_hash_functions = generate_hash_function(k,vector_dimensions,non_zero_values)
# random_hash_functions.to_csv('LSH Search Results/LSH Random Hash Functions.csv')

# Import Hash functions *
random_hash_functions_df = pd.read_json('LSH Search Results/LSH Random Hash Functions.json')

# # Do LSH Calculations for database *
# df['Binary Vectors'] = df['Vector'].apply( lambda x : generate_binary_vectors(x,random_hash_functions_df))
# df.to_csv('LSH Search Results/LSH Database with Binary Vectors.csv')

# Do LSH Calculations for query vectors *
query_dict_binary_vectors = [generate_binary_vectors(vector, random_hash_functions_df) for vector in query_matrix]

query_df['Vectors'] = [row.tolist() for row in query_matrix]
query_df['Binary Vectors'] = query_dict_binary_vectors

query_df.to_csv('LSH Search Results/LSH Query Vectors with Binary Vectors.csv')

In [7]:
import json

query_df = pd.read_csv('LSH Search Results/LSH Query Vectors with Binary Vectors.csv')
df = pd.read_csv('LSH Search Results/LSH Database with Binary Vectors.csv')

query_df['Binary Vectors'] = query_df['Binary Vectors'].apply(lambda x : json.loads(x))
df['Binary Vectors'] = df['Binary Vectors'].apply(lambda x : json.loads(x))

In [21]:
query_df.head()

,Unnamed: 0,Query Text,Vectors,Binary Vectors
0,Q1,resilient investment banker,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, ..."
1,Q2,2 years experience product manager,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, ..."
2,Q3,10 years risk analyst problem solver,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, ..."
3,Q4,tax analyst for big company,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, ..."
4,Q5,software engineer for google or amazon,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, ..."


In [25]:
import time

# Specify the folder path
folder_path = 'LSH Search Results'

# Define the Excel file path
excel_file_path = os.path.join(folder_path, 'LSH_Search_Results.xlsx')

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:     
    
    for query_name,query_row in query_df.iterrows():

        mismatched_list = []

        start_time = time.time()
        for i, row in df.iterrows():
            mismatched_list.append(hamming_distance(query_row['Binary Vectors'],row['Binary Vectors'])) 
        end_time = time.time()
        print("Duration taken to calculate hamming distance of '"+ str(query_name) +"' with dataset:", end_time-start_time)
        
        df['Distance'] = mismatched_list

        # Get only data and distance from query_name
        query_result_df = df[['Data', 'Distance']].copy()

        # Get the top 50 results based on the 'Distance' column
        # query_result_top = query_result_df.nsmallest(50, 'Distance') 
    
        # Loop through each unique query and create separate sheets
        query_result_df.to_excel(writer, sheet_name=f'Results_{query_name}', index=False)

Duration taken to calculate hamming distance of '0' with dataset: 0.29659366607666016
Duration taken to calculate hamming distance of '1' with dataset: 0.30701422691345215
Duration taken to calculate hamming distance of '2' with dataset: 0.30878257751464844
Duration taken to calculate hamming distance of '3' with dataset: 0.30289363861083984
Duration taken to calculate hamming distance of '4' with dataset: 0.3064234256744385
Duration taken to calculate hamming distance of '5' with dataset: 0.29548072814941406
Duration taken to calculate hamming distance of '6' with dataset: 0.30573081970214844
Duration taken to calculate hamming distance of '7' with dataset: 0.2996821403503418
Duration taken to calculate hamming distance of '8' with dataset: 0.31114745140075684
Duration taken to calculate hamming distance of '9' with dataset: 0.3057389259338379
